In [1]:
import pandas as pd
import numpy as np
import solver as sl
import matplotlib.pyplot as plt
import itertools as itter
import heapq
import networkx as nx

from map import Map
from cluster import Cluster
from distance import *

%load_ext autoreload
%autoreload 2

In [78]:
map = Map(seed=10)
map.create_random_nodes(n=1000)
map.create_net_by_voronoi()

# fig, ax = plt.subplots(1, 2, figsize=(10,5))
# map.plot(node_label='index',ax=ax[0])
# map.plot(node_label='demand',ax=ax[1])
# _ = ax[0].set_title('IDs')
# _ = ax[1].set_title('Demands')

In [81]:
connectivity = nx.to_pandas_adjacency(map.network)
connectivity = np.array(connectivity)

net = nx.Graph()
for n in map.network.nodes():
    net.add_node(Cluster(n))

ordered = []
for c in itter.combinations(net.nodes(), r=2):
    n1, n2 = c[0], c[1]
    if (connectivity[n1.index, n2.index]):
        # diff = np.abs(n1.total_demand - n2.total_demand)
        net.add_edge(n1, n2)


def find_max_diff(net: nx.Graph):
    max_diff = 0
    max_edge = []
    for ed in net.edges():
        c1, c2 = ed[0], ed[1]
        diff = np.abs(c1.total_demand - c2.total_demand)
        if diff > max_diff:
            max_diff = diff
            max_edge = ed
    return max_edge


print(find_max_diff(net))

# print(len(net))
# for c in net.nodes():
#     print(f"id: {c.index}, td: {c.total_demand}, nodes: {c.nodes}")

while net.order()>3:
    max_edge = find_max_diff(net)
    c1, c2 = max_edge[0], max_edge[1]
    c1.merge(c2)
    net = nx.contracted_nodes(net, c1, c2, self_loops=False)
# print("\n", len(net.edges()))
# for c in net.nodes():
#     print(f"id: {c.index}, mean: {c.mean_demand}, nodes: {c.nodes}")

for c in net.nodes():
    print(f"id: {c.index}, td: {c.total_demand}, mean: {c.mean_demand}, nodes: {len(c.nodes)}")

(<cluster.Cluster object at 0x000002306CABF890>, <cluster.Cluster object at 0x000002306DA8F850>)
id: 0, td: 50554, mean: 50.80804020100503, nodes: 995
id: 444, td: 106, mean: 53.0, nodes: 2
id: 498, td: 110, mean: 36.666666666666664, nodes: 3
